In [1]:
!pip install pymongo

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [146]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

from bs4 import BeautifulSoup as bs
import requests
from pprintpp import pprint as pp
import pandas as pd
import numpy as np
from time import sleep

*Для выполнения задания воспользуемся парсингом страниц hh.ru, который мы выполнили в предыдущей работе.

In [122]:
#функция для парсинга сайта hh.ru, принимает аргументы название вакансии и количество страниц, с которых будут поступать данные
def vacancy(name, num_pages):
    url = f'https://reutov.hh.ru/search/vacancy?area=1&search_field=name&search_field=company_name&search_field=description&enable_snippets=true&text={name}&page='
    headers = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0'}
    vacancy_list = []
    for page_number in range(num_pages):
        url_page = url + str(page_number)
        res = requests.get(url_page, headers = headers)
        soup = bs(res.content, 'html.parser')
        #Поскольку не во всех вакансиях присутствует тег с информацией о зарплате, рассмотрим отдельные списки вакансий, в которых зарплата присутствует и в которых нет
        #Наименование вакансии
        vacancy_name = soup.find_all('a', {'class':'serp-item__title'})
        vacancy_name_with_salary = [soup.find_all('span', {'class':'bloko-header-section-3'})[i].parent.find('a', {'class':'serp-item__title'}) for i in range(len(soup.find_all('span', {'class':'bloko-header-section-3'})))]
        vacancy_name_no_salary = [vacancy_name[i] for i in range(len(vacancy_name)) if vacancy_name[i].parent.parent.parent.find_all('span', {'class':'bloko-header-section-3'}) == []]
        #Компания
        company = soup.find_all('a', {'class':'bloko-link bloko-link_kind-tertiary'})
        company_with_salary = [soup.find_all('span', {'class':'bloko-header-section-3'})[i].parent.parent.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}) for i in range(len(soup.find_all('span', {'class':'bloko-header-section-3'})))]
        company_no_salary = company_no_salary = [company[i] for i in range(len(company)) if vacancy_name[i].parent.parent.parent.find_all('span', {'class':'bloko-header-section-3'}) == []]
        #Зарплата
        salary = soup.find_all('span', {'class':'bloko-header-section-3'})

        vacancy_list_page = []

        for i in range(len(vacancy_name_with_salary)):
            if 'от' in salary[i].text:
                salary_min = int(salary[i].text.replace('от ', '').replace('\u202f', '').split(' ')[0])
                salary_max = 'not stated'
                currency = salary[i].text.split(' ')[2]
            elif 'до' in salary[i].text:
                salary_min = 'not stated'
                salary_max = int(salary[i].text.replace('до ', '').replace('\u202f', '').split(' ')[0])
                currency = salary[i].text.split(' ')[2]                            
            else: 
                salary_min = int(salary[i].text.split(' ')[0].replace('\u202f', ''))
                salary_max = int(salary[i].text.split(' ')[2].replace('\u202f', ''))
                currency = salary[i].text.split(' ')[3]
            vacancy_list_page.append({'name': vacancy_name_with_salary[i].text, 'company': company_with_salary[i].text, 'salary_min':salary_min, 'salary_max':salary_max, 'salary_currency':currency, 'link': vacancy_name_with_salary[i]['href']})
        for i in range(len(vacancy_name_no_salary)):
            vacancy_list_page.append({'name':vacancy_name_no_salary[i].text, 'company': company_no_salary[i].text, 'salary_min':'not stated', 'salary_max':'not stated', 'salary_currency': 'not stated', 'link':vacancy_name_no_salary[i]['href']})
        vacancy_list = vacancy_list + vacancy_list_page
        #vacancy_list.append(vacancy_list_page)
        sleep(3)
    return vacancy_list  

In [123]:
client = MongoClient()
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [124]:
db = client['vacancy']

In [125]:
#функция для внесения данных с сайта hh.ru в базу данных MongoDB "vacancy"
def insert_into_vacancy_db():
    #запрашиваем у пользователя данные по названию вакансии и количеству страниц, с которых будут предоставлены данные
    name = input('Введите название вакансии: ')
    num_pages = int(input('Введите количество страниц, с которых будет собрана информация: '))
    vacancy_list = vacancy(name, num_pages)
    v_info = db[name]
    for el in vacancy_list:
        element_in_db = v_info.find(el)
        element_in_db_list = []
        for el in element_in_db:
            element_in_db_list.append(el)
        if element_in_db_list == []:
            v_info.insert_one(el)
    count_docs = v_info.find()
    num = 0
    for i in count_docs:
        num += 1
    return(f'В базе данных "vacancy" в коллекции {name} после внесения новых вакансий всего {num} вакансий.')    

In [126]:
insert_into_vacancy_db()

Введите название вакансии: data
Введите количество страниц, с которых будет собрана информация: 1


'В базе данных "vacancy" в коллекции data после внесения новых вакансий всего 20 вакансий.'

In [127]:
db.list_collection_names() #проверяем коллекции

['data', 'java', 'python']

In [128]:
print(client.list_database_names()) #проверяем базы данных

['admin', 'config', 'local', 'vacancy']


2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

Для целей решения данной задачи предположим, что функция выводит информацию следубщим образом:
    1. В случае, если есть минимальный и максимальный порог заработной платы, выводятся вакансии, где максимальный порог больше введенной пользователем суммы
    2. В случае, если есть только максимальный порог, выводятся вакансии с зарплатой больше него
    3. В случае, если есть только минимальный порог, выводятся вакансии с зарплатой больше него
    4. В случае, если зарплата не указана, эти вакансии не выводятся

In [154]:
def salary_gt():
    name = input('Введите название вакансии: ')
    collection = db[name]
    salary = int(input('Введите заработную плату: '))
    salary_gt_list = [collection.find({'salary_min':{'$gt':salary}, 'salary_max':'not stated'}), collection.find({'salary_max':{'$gt':salary}, 'salary_min':'not stated'}), collection.find({'salary_max':{'$gt':salary}, 'salary_min':{'$ne':'not stated'}})]
    for i in range(len(salary_gt_list)):
        for el in salary_gt_list[i]:
            pp(el)

In [158]:
salary_gt()

Введите название вакансии: java
Введите заработную плату: 250000
{
    '_id': ObjectId('63f533f7d695315b37a2a431'),
    'company': 'ООО\xa0HeadHunter::Backend',
    'link': 'https://reutov.hh.ru/vacancy/76713727?from=vacancy_search_list&query=java',
    'name': 'Java разработчик в команду "Навыки"',
    'salary_currency': 'руб.',
    'salary_max': 'not stated',
    'salary_min': 350000,
}
{
    '_id': ObjectId('63f533f7d695315b37a2a435'),
    'company': 'АО\xa0НТЦ ЕЭС Информационные Комплексы',
    'link': 'https://reutov.hh.ru/vacancy/76643997?from=vacancy_search_list&query=java',
    'name': 'Ведущий Java разработчик (удалённо)',
    'salary_currency': 'руб.',
    'salary_max': 400000,
    'salary_min': 350000,
}
{
    '_id': ObjectId('63f533f7d695315b37a2a436'),
    'company': 'BrainHub',
    'link': 'https://reutov.hh.ru/vacancy/76610003?from=vacancy_search_list&query=java',
    'name': 'Senior Java developer',
    'salary_currency': 'руб.',
    'salary_max': 450000,
    'salary_mi